In [1]:
import tensorflow as tf
import numpy as np

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

tf.executing_eagerly()

c:\users\windows\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Default GPU Device: /device:GPU:0


True

In [2]:
data = np.load('./helpers/data.npy',allow_pickle=True)

In [3]:
my_dict = dict()
for d in data:
    if d.shape[0] not in my_dict:
        my_dict[d.shape[0]] = []
    my_dict[d.shape[0]].append(d)
    
for k in my_dict:
    my_dict[k] = np.stack(my_dict[k])
    print(my_dict[k].shape)

(3, 4, 4)
(2, 5, 4)
(7, 7, 4)
(43, 8, 4)
(35, 9, 4)
(18, 10, 4)
(228, 11, 4)
(506, 12, 4)
(628, 13, 4)
(431, 14, 4)
(1870, 15, 4)
(4256, 16, 4)
(7292, 17, 4)
(5418, 19, 4)
(37133, 20, 4)
(42408, 23, 4)


In [4]:
indexes = np.argmax(my_dict[23][0],axis=1)
indexes

array([1, 1, 1, 2, 1, 0, 2, 1, 1, 0, 2, 0, 0, 0, 2, 0, 1, 0, 2, 0, 2, 0,
       0], dtype=int64)

In [5]:
from helpers.cfg import CFGHelper
from helpers.dataset_generator import sample_grammar

cfg = CFGHelper(sample_grammar)
tokens = cfg.indexes_to_tokens(indexes)
' '.join(tokens)

'( x + x ) + x + x + x - y'

In [6]:
batch_size = 5
latent_dim = 100
vocab_size = 4
seq_len = 20

In [7]:
class CFGNetEncoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, seq_len, latent_dim):
        super(CFGNetEncoder, self).__init__()
        
        self.seq_len = seq_len
        self.vocab_size = vocab_size
        self.latent_dim = latent_dim
        
        input_shape = (None, self.seq_len * self.vocab_size)
        self.dense1 = tf.keras.layers.Dense(vocab_size * seq_len, input_shape=input_shape)
        self.dense2 = tf.keras.layers.Dense(self.latent_dim)

    def build(self, input_shape):
        super(CFGNetEncoder, self).build(input_shape)
        
    def call(self, v):
        v = tf.reshape(v,(-1, self.seq_len * self.vocab_size))
        v = self.dense1(v)
        v = self.dense2(v)
        return v
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.latent_dim)

# temp_encoder = CFGNetEncoder(vocab_size, seq_len, latent_dim)
# y = tf.zeros((batch_size, seq_len, vocab_size))
# state = temp_encoder(y)
# state.shape
# temp_encoder.trainable_variables

In [8]:
class CFGNetDecoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, seq_len, latent_dim):
        super(CFGNetDecoder, self).__init__()
        
        self.seq_len = seq_len
        self.vocab_size = vocab_size
        self.latent_dim = latent_dim
        
        input_shape = (None, latent_dim)
        self.dense1 = tf.keras.layers.Dense(self.latent_dim, input_shape=input_shape)
        self.dense2 = tf.keras.layers.Dense(self.vocab_size * self.seq_len,activation='softmax') 
        
    def build(self, input_shape):
        super(CFGNetDecoder, self).build(input_shape)
        
    def call(self, v):
        v = self.dense1(v)
        v = self.dense2(v)
        v = tf.reshape(v, (-1, self.seq_len, self.vocab_size))
        return v
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.seq_len, self.vocab_size)

# temp_decoder = CFGNetDecoder(vocab_size, seq_len, latent_dim)
# state = tf.zeros((batch_size, latent_dim))
# out = temp_decoder(state)
# out.shape

In [9]:
class CFGNetAutoencoder(tf.keras.Model):
    def __init__(self, vocab_size, seq_len, latent_dim):
        super(CFGNetAutoencoder, self).__init__()
        
        self.seq_len = seq_len
        self.vocab_size = vocab_size
        self.latent_dim = latent_dim
        self.encoder = CFGNetEncoder(self.vocab_size, self.seq_len, self.latent_dim)
        self.decoder = CFGNetDecoder(self.vocab_size, self.seq_len, self.latent_dim)
        
    def build(self, input_shape):
        super(CFGNetAutoencoder, self).build(input_shape)

    def call(self, v):
        seq_len = tf.shape(v)[1]
        state = self.encoder(v)
        return self.decoder(state)
    
    def compute_output_shape(self, input_shape):
        return input_shape

# temp_autoencoder = CFGNetAutoencoder(vocab_size, seq_len, latent_dim)
# v = tf.zeros((batch_size, seq_len, vocab_size))
# out = temp_autoencoder(v)
# print(out.shape)
# temp_autoencoder.summary()
# temp_autoencoder.trainable_variables

In [10]:
# def custom_loss(y, y_):
#     return tf.reduce_mean(y-y_)

# y = tf.zeros((batch_size, seq_len, vocab_size))
# y_ = tf.ones((batch_size, seq_len, vocab_size))
# with tf.Session() as sess:
#     print(sess.run(custom_loss(y,y_)))

In [11]:
key = 4

In [12]:
data_to_use = my_dict[key]
data_to_use.shape

(3, 4, 4)

In [13]:
cfg_net_autoencoder = CFGNetAutoencoder(data_to_use.shape[2], data_to_use.shape[1], latent_dim)
# loss=keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=False, label_smoothing=0)
cfg_net_autoencoder.compile(loss='categorical_crossentropy', optimizer='adam')
# cfg_net_autoencoder.compile(loss='categorical_crossentropy', optimizer='sgd')

In [14]:
import os
import datetime

# for key in my_dict:
for key in [key]:
    data_to_use = my_dict[key]
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs_ff')
    cfg_net_autoencoder.fit(x=data_to_use,y=data_to_use,batch_size=20000,epochs=1000, callbacks=[tensorboard_callback])
#     cfg_net_autoencoder.fit(x=data_to_use,y=data_to_use,batch_size=20000,epochs=100)

import winsound
frequency = 2500 
duration = 500 
winsound.Beep(frequency, duration)

Train on 3 samples
Epoch 1/1000
3/3 [==============================] - 2s 622ms/sample - loss: 1.4883
Epoch 2/1000
3/3 [==============================] - 0s 117ms/sample - loss: 1.3392
Epoch 3/1000
3/3 [==============================] - 0s 13ms/sample - loss: 1.2016
Epoch 4/1000
3/3 [==============================] - 0s 13ms/sample - loss: 1.0750
Epoch 5/1000
3/3 [==============================] - 0s 12ms/sample - loss: 0.9584
Epoch 6/1000
3/3 [==============================] - 0s 12ms/sample - loss: 0.8511
Epoch 7/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.7530
Epoch 8/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.6637
Epoch 9/1000
3/3 [==============================] - 0s 16ms/sample - loss: 0.5833
Epoch 10/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.5117
Epoch 11/1000
3/3 [==============================] - 0s 12ms/sample - loss: 0.4486
Epoch 12/1000
3/3 [==============================] - 0s 12ms/sample - loss:

3/3 [==============================] - 0s 13ms/sample - loss: 0.0026
Epoch 99/1000
3/3 [==============================] - 0s 12ms/sample - loss: 0.0026
Epoch 100/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.0025
Epoch 101/1000
3/3 [==============================] - 0s 12ms/sample - loss: 0.0025
Epoch 102/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.0024
Epoch 103/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.0024
Epoch 104/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.0023
Epoch 105/1000
3/3 [==============================] - 0s 16ms/sample - loss: 0.0023
Epoch 106/1000
3/3 [==============================] - 0s 16ms/sample - loss: 0.0023
Epoch 107/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.0022
Epoch 108/1000
3/3 [==============================] - 0s 19ms/sample - loss: 0.0022
Epoch 109/1000
3/3 [==============================] - 0s 13ms/sample - loss: 0.0022
Epoch 11

Epoch 195/1000
3/3 [==============================] - 0s 13ms/sample - loss: 7.9933e-04
Epoch 196/1000
3/3 [==============================] - 0s 13ms/sample - loss: 7.9237e-04
Epoch 197/1000
3/3 [==============================] - 0s 12ms/sample - loss: 7.8544e-04
Epoch 198/1000
3/3 [==============================] - 0s 13ms/sample - loss: 7.7861e-04
Epoch 199/1000
3/3 [==============================] - 0s 12ms/sample - loss: 7.7190e-04
Epoch 200/1000
3/3 [==============================] - 0s 16ms/sample - loss: 7.6528e-04
Epoch 201/1000
3/3 [==============================] - 0s 16ms/sample - loss: 7.5874e-04
Epoch 202/1000
3/3 [==============================] - 0s 14ms/sample - loss: 7.5229e-04
Epoch 203/1000
3/3 [==============================] - 0s 15ms/sample - loss: 7.4590e-04
Epoch 204/1000
3/3 [==============================] - 0s 15ms/sample - loss: 7.3960e-04
Epoch 205/1000
3/3 [==============================] - 0s 13ms/sample - loss: 7.3339e-04
Epoch 206/1000
3/3 [============

3/3 [==============================] - 0s 13ms/sample - loss: 4.0502e-04
Epoch 289/1000
3/3 [==============================] - 0s 12ms/sample - loss: 4.0252e-04
Epoch 290/1000
3/3 [==============================] - 0s 12ms/sample - loss: 4.0006e-04
Epoch 291/1000
3/3 [==============================] - 0s 13ms/sample - loss: 3.9765e-04
Epoch 292/1000
3/3 [==============================] - 0s 12ms/sample - loss: 3.9523e-04
Epoch 293/1000
3/3 [==============================] - 0s 20ms/sample - loss: 3.9286e-04
Epoch 294/1000
3/3 [==============================] - 0s 18ms/sample - loss: 3.9051e-04
Epoch 295/1000
3/3 [==============================] - 0s 13ms/sample - loss: 3.8815e-04
Epoch 296/1000
3/3 [==============================] - 0s 15ms/sample - loss: 3.8582e-04
Epoch 297/1000
3/3 [==============================] - 0s 12ms/sample - loss: 3.8352e-04
Epoch 298/1000
3/3 [==============================] - 0s 12ms/sample - loss: 3.8127e-04
Epoch 299/1000
3/3 [===========================

Epoch 382/1000
3/3 [==============================] - 0s 13ms/sample - loss: 2.4462e-04
Epoch 383/1000
3/3 [==============================] - 0s 16ms/sample - loss: 2.4344e-04
Epoch 384/1000
3/3 [==============================] - 0s 12ms/sample - loss: 2.4231e-04
Epoch 385/1000
3/3 [==============================] - 0s 13ms/sample - loss: 2.4115e-04
Epoch 386/1000
3/3 [==============================] - 0s 12ms/sample - loss: 2.4004e-04
Epoch 387/1000
3/3 [==============================] - 0s 12ms/sample - loss: 2.3894e-04
Epoch 388/1000
3/3 [==============================] - 0s 16ms/sample - loss: 2.3783e-04
Epoch 389/1000
3/3 [==============================] - 0s 13ms/sample - loss: 2.3674e-04
Epoch 390/1000
3/3 [==============================] - 0s 15ms/sample - loss: 2.3563e-04
Epoch 391/1000
3/3 [==============================] - 0s 13ms/sample - loss: 2.3453e-04
Epoch 392/1000
3/3 [==============================] - 0s 12ms/sample - loss: 2.3345e-04
Epoch 393/1000
3/3 [============

...

3/3 [==============================] - 0s 14ms/sample - loss: 1.1811e-04
Epoch 570/1000
3/3 [==============================] - 0s 15ms/sample - loss: 1.1775e-04
Epoch 571/1000
3/3 [==============================] - 0s 52ms/sample - loss: 1.1733e-04
Epoch 572/1000
3/3 [==============================] - 0s 12ms/sample - loss: 1.1697e-04
Epoch 573/1000
3/3 [==============================] - 0s 13ms/sample - loss: 1.1660e-04
Epoch 574/1000
3/3 [==============================] - 0s 12ms/sample - loss: 1.1622e-04
Epoch 575/1000
3/3 [==============================] - 0s 12ms/sample - loss: 1.1585e-04
Epoch 576/1000
3/3 [==============================] - 0s 13ms/sample - loss: 1.1545e-04
Epoch 577/1000
3/3 [==============================] - 0s 12ms/sample - loss: 1.1509e-04
Epoch 578/1000
3/3 [==============================] - 0s 12ms/sample - loss: 1.1475e-04
Epoch 579/1000
3/3 [==============================] - 0s 13ms/sample - loss: 1.1435e-04
Epoch 580/1000
3/3 [===========================

Epoch 663/1000
3/3 [==============================] - 0s 19ms/sample - loss: 8.8863e-05
Epoch 664/1000
3/3 [==============================] - 0s 13ms/sample - loss: 8.8575e-05
Epoch 665/1000
3/3 [==============================] - 0s 15ms/sample - loss: 8.8351e-05
Epoch 666/1000
3/3 [==============================] - 0s 13ms/sample - loss: 8.8088e-05
Epoch 667/1000
3/3 [==============================] - 0s 13ms/sample - loss: 8.7825e-05
Epoch 668/1000
3/3 [==============================] - 0s 13ms/sample - loss: 8.7596e-05
Epoch 669/1000
3/3 [==============================] - 0s 15ms/sample - loss: 8.7358e-05
Epoch 670/1000
3/3 [==============================] - 0s 13ms/sample - loss: 8.7104e-05
Epoch 671/1000
3/3 [==============================] - 0s 12ms/sample - loss: 8.6866e-05
Epoch 672/1000
3/3 [==============================] - 0s 13ms/sample - loss: 8.6632e-05
Epoch 673/1000
3/3 [==============================] - 0s 16ms/sample - loss: 8.6379e-05
Epoch 674/1000
3/3 [============

Epoch 757/1000
3/3 [==============================] - 0s 16ms/sample - loss: 6.9176e-05
Epoch 758/1000
3/3 [==============================] - 0s 16ms/sample - loss: 6.9037e-05
Epoch 759/1000
3/3 [==============================] - 0s 12ms/sample - loss: 6.8888e-05
Epoch 760/1000
3/3 [==============================] - 0s 33ms/sample - loss: 6.8649e-05
Epoch 761/1000
3/3 [==============================] - 0s 13ms/sample - loss: 6.8505e-05
Epoch 762/1000
3/3 [==============================] - 0s 16ms/sample - loss: 6.8361e-05
Epoch 763/1000
3/3 [==============================] - 0s 13ms/sample - loss: 6.8142e-05
Epoch 764/1000
3/3 [==============================] - 0s 16ms/sample - loss: 6.8013e-05
Epoch 765/1000
3/3 [==============================] - 0s 13ms/sample - loss: 6.7834e-05
Epoch 766/1000
3/3 [==============================] - 0s 16ms/sample - loss: 6.7661e-05
Epoch 767/1000
3/3 [==============================] - 0s 14ms/sample - loss: 6.7472e-05
Epoch 768/1000
3/3 [============

3/3 [==============================] - 0s 21ms/sample - loss: 5.5485e-05
Epoch 851/1000
3/3 [==============================] - 0s 16ms/sample - loss: 5.5376e-05
Epoch 852/1000
3/3 [==============================] - 0s 16ms/sample - loss: 5.5241e-05
Epoch 853/1000
3/3 [==============================] - 0s 16ms/sample - loss: 5.5127e-05
Epoch 854/1000
3/3 [==============================] - 0s 17ms/sample - loss: 5.5008e-05
Epoch 855/1000
3/3 [==============================] - 0s 15ms/sample - loss: 5.4879e-05
Epoch 856/1000
3/3 [==============================] - 0s 16ms/sample - loss: 5.4750e-05
Epoch 857/1000
3/3 [==============================] - 0s 17ms/sample - loss: 5.4630e-05
Epoch 858/1000
3/3 [==============================] - 0s 22ms/sample - loss: 5.4521e-05
Epoch 859/1000
3/3 [==============================] - 0s 14ms/sample - loss: 5.4397e-05
Epoch 860/1000
3/3 [==============================] - 0s 19ms/sample - loss: 5.4278e-05
Epoch 861/1000
3/3 [===========================

Epoch 944/1000
3/3 [==============================] - 0s 13ms/sample - loss: 4.5351e-05
Epoch 945/1000
3/3 [==============================] - 0s 16ms/sample - loss: 4.5252e-05
Epoch 946/1000
3/3 [==============================] - 0s 13ms/sample - loss: 4.5167e-05
Epoch 947/1000
3/3 [==============================] - 0s 20ms/sample - loss: 4.5063e-05
Epoch 948/1000
3/3 [==============================] - 0s 16ms/sample - loss: 4.4959e-05
Epoch 949/1000
3/3 [==============================] - 0s 20ms/sample - loss: 4.4889e-05
Epoch 950/1000
3/3 [==============================] - 0s 12ms/sample - loss: 4.4805e-05
Epoch 951/1000
3/3 [==============================] - 0s 16ms/sample - loss: 4.4705e-05
Epoch 952/1000
3/3 [==============================] - 0s 13ms/sample - loss: 4.4631e-05
Epoch 953/1000
3/3 [==============================] - 0s 16ms/sample - loss: 4.4526e-05
Epoch 954/1000
3/3 [==============================] - 0s 12ms/sample - loss: 4.4442e-05
Epoch 955/1000
3/3 [============

In [15]:
predictions = cfg_net_autoencoder.predict(data_to_use)

In [16]:
np.argmax(data_to_use,axis=1)

array([[0, 3, 2, 0],
       [0, 0, 2, 0],
       [0, 0, 2, 3]], dtype=int64)

In [17]:
one_hots = np.argmax(predictions,axis=1)
one_hots

array([[0, 3, 2, 3],
       [1, 3, 3, 3],
       [0, 3, 2, 3]], dtype=int64)